In [1]:
# load Data
import pandas as pd
import numpy as np

data_wfh = pd.read_csv('wfh.csv')
data_quit = pd.read_csv('quit_data.csv')
data_pefm = pd.read_csv('performance.csv')

# Handle the Non-binary Variables
data_quit['role_dummy'] = (data_quit['role'] == 'Manager').astype(int)
data_quit['male_dummy'] = (data_quit['male'] == 'Male').astype(int)
data_quit['far_dummy'] = (data_quit['live_far'] == ' Commute >  90 mins').astype(int)

print(data_wfh.head())
print(data_wfh.describe())

print(data_quit.head())
print(data_quit.describe())

print(data_pefm.head())
print(data_pefm.describe())

   treat  volunteer  takeuprate  date
0      0          0    0.004016     0
1      1          0    0.454183     0
2      0          1    0.001887     0
3      1          1    0.269450     0
4      0          0    0.004032     2
            treat   volunteer  takeuprate        date
count  120.000000  120.000000  120.000000  120.000000
mean     0.500000    0.500000    0.251610   55.833333
std      0.502096    0.502096    0.262997   32.137398
min      0.000000    0.000000    0.000000    0.000000
25%      0.000000    0.000000    0.003906   30.000000
50%      0.500000    0.500000    0.142388   57.000000
75%      1.000000    1.000000    0.498012   84.000000
max      1.000000    1.000000    0.826772  107.000000
        treat         role  male             live_far  quit  role_dummy  \
0   Treatment      Manager  Male   Commute <= 90 mins     0           1   
1   Treatment      Manager  Male   Commute >  90 mins     0           1   
2     Control  Non-manager  Male   Commute <= 90 mins     0  

# 1. Validation
## Problem of Confounding

The experiment used randomized assignment based on odd-numbered birthdays for the treatment group (HWFH) and even-numbered birthdays for the control group. Randomization ensures that, on average, pre-treatment characteristics like **gender, commute distance, role** that are not correlated with birthdays are balanced between the treatment and control groups, solving the problem of confounding.

A potential confounder could be something correlated with odd/even birthdays that influences quitting or performance. However, odd/even birthday assignment is effectively random and unlikely to be systematically related to factors like job performance or quit rates.

Below are the balance tables showing the randomization is successful. **role_dummy, male_dummy, and far_dummy** are similar between groups.

In [2]:
# validation for randomization
balance_table = data_quit.groupby('treat').agg(
    role_mean=('role_dummy', 'mean'),
    role_std=('role_dummy', 'std'),
    male_mean=('male_dummy', 'mean'),
    male_std=('male_dummy', 'std'),
    live_far_mean=('far_dummy', 'mean'),
    live_far_std=('far_dummy', 'std'),
    quit_cnt=('treat', 'count'),
)

balance_table

role_mean  role_std  male_mean  male_std  live_far_mean  \
treat                                                                 
 Treatment   0.242424  0.428820   0.637626  0.480990       0.392677   
Control      0.247561  0.431859   0.654878  0.475698       0.365854   

            live_far_std  quit_cnt  
treat                               
 Treatment      0.488654       792  
Control         0.481963       820

In [3]:
data_pefm.groupby('treat').agg(
    cnt=('treat','count')
)

cnt
treat     
0      681
1      674

## Intervention Affect WFH Rates?

By looking at the tot and its 95% confidence interval for the takeup rate for working from home, we can see that the treatment have significant positive effect on the takeup rate for working from home. The average ITT is 49.6% and the average TOT is 52.1%, which is even higher.

In [4]:
itt_table = data_wfh.groupby('treat')['takeuprate'].agg(
    mean = 'mean',
    std = 'std',
    count = 'count'
)

itt_table['itt'] = itt_table['mean'] - itt_table['mean'][0]
itt_table['se_mean'] = itt_table['std']/np.sqrt(itt_table['count'])
itt_table['itt_se'] = np.sqrt(itt_table['se_mean']**2 + itt_table['se_mean'][0]**2)
itt_table['lb'] = itt_table['itt'] - 1.96*itt_table['itt_se']
itt_table['ub'] = itt_table['itt'] + 1.96*itt_table['itt_se']

print(itt_table[['itt', 'lb', 'ub']][1:2])

            itt        lb        ub
treat                              
1      0.496166  0.465876  0.526457


# 2 Effect of HWFH on Quit Rate and Performance
## HWFH: quit rate

In [5]:
itt_table = data_quit.groupby('treat')['quit'].agg(
    mean = 'mean',
    std = 'std',
    count = 'count'
)

itt_table['itt'] = itt_table['mean'] - itt_table['mean'].iloc[1]
itt_table['se_mean'] = itt_table['std']/np.sqrt(itt_table['count'])
itt_table['itt_se'] = np.sqrt(itt_table['se_mean']**2 + itt_table['se_mean'].iloc[1]**2)
itt_table['lb'] = itt_table['itt'] - 1.96*itt_table['itt_se']
itt_table['ub'] = itt_table['itt'] + 1.96*itt_table['itt_se']

print(itt_table[['itt', 'lb', 'ub']][0:1])
compliance_rate = data_wfh[data_wfh['treat'] == 1]['takeuprate'].mean()
itt_table['tot'] = itt_table['itt']/compliance_rate
itt_table['tot_se'] = itt_table['itt_se'] / compliance_rate
itt_table['tot_lb'] = itt_table['tot'] - 1.96*itt_table['tot_se']
itt_table['tot_ub'] = itt_table['tot'] + 1.96*itt_table['tot_se']

print(itt_table[['tot', 'tot_lb', 'tot_ub']][0:1])

                itt       lb        ub
treat                                 
Treatment -2.397142 -4.71026 -0.084024
                tot    tot_lb    tot_ub
treat                                  
Treatment -4.797224 -9.426296 -0.168152


This result indicates that employees assigned with the HWFH option are, on average, 4.8% less likely to quit compared to the control group, the effect is statistically significant at the 5% level.

# HWFH: performance

In [6]:
data_pefm['perf_2022h1'].unique()

array(['B', 'A', 'D', 'B+', 'C'], dtype=object)

In [7]:
# Assign 4 to A, 3.5 to B+, 3 to B, 2 to C, 1 to D
# Any mapping would be correct as long as the ordinality is kept.
perf_mapping = {
    "A": 4,
    "B+": 3.5,
    "B": 3,
    "C": 2,
    "D": 1
}
data_pefm['performance'] = data_pefm['perf_2022h1'].map(perf_mapping)
itt_table = data_pefm.groupby('treat')['performance'].agg(
    mean = 'mean',
    std = 'std',
    count = 'count'
)

itt_table['itt'] = itt_table['mean'] - itt_table['mean'].iloc[0]
itt_table['se_mean'] = itt_table['std']/np.sqrt(itt_table['count'])
itt_table['itt_se'] = np.sqrt(itt_table['se_mean']**2 + itt_table['se_mean'].iloc[0]**2)
itt_table['lb'] = itt_table['itt'] - 1.96*itt_table['itt_se']
itt_table['ub'] = itt_table['itt'] + 1.96*itt_table['itt_se']

print(itt_table[['itt', 'lb', 'ub']][1:2])

compliance_rate = data_wfh[data_wfh['treat'] == 1]['takeuprate'].mean()
itt_table['tot'] = itt_table['itt']/compliance_rate
itt_table['tot_se'] = itt_table['itt_se'] / compliance_rate
itt_table['tot_lb'] = itt_table['tot'] - 1.96*itt_table['tot_se']
itt_table['tot_ub'] = itt_table['tot'] + 1.96*itt_table['tot_se']

print(itt_table[['tot', 'tot_lb', 'tot_ub']][1:2])

            itt        lb       ub
treat                             
1     -0.029957 -0.079433  0.01952
           tot    tot_lb    tot_ub
treat                             
1     -0.05995 -0.158964  0.039064


The average ITT estimate is −0.03, and the average TOT estimate is −0.06. Both 95% confidence intervals include 0, indicating a noisy estimate and no significant effect of HWFH on performance. **TOT needs to be used for estimating the effect of HWFH on working performance.**

# 3 Excludability condition

Excludability condition: Only way assignment to treatment group (i.e., offer of treatment) can affect average outcomes is through the people who take treatment. Workers not assigned to the WFH might get offended for being excluded which could ultimately affect their performance.

# 4 Attrition

If an employee quits, the quit variable will be a 0 instead of missing number, so no worries about attrition here. If an employee quits, the performance variable will be left blank, and thus we should be concerned about differential attrition here. In our case attrition is defined as missing performance since it is our outcome metric, and thus missing pre-treatment variables is not considered as the attrition in this setting.

# 5 Summary

The results of this experiment indicate that hybrid working from home (HWFH) significantly increases the quitting rate but has no significant effect on performance. Despite the increased quitting rate, I recommend scaling up HWFH.

To better understand the heterogeneity of effects across subgroups and identify underlying causes, future experiments should collect additional information such as employees' job types (e.g., tech, service), mental health conditions (to explore potential reasons for quitting), and direct feedback through surveys about their preferences for remote vs. on-site work. This approach will provide a more comprehensive basis for informed decision-making.

# 6 Lee bounds trimming approach

In [8]:
# calculate attrition rate for performance
attr_pefm = data_pefm['perf_2022h1'].count()/data_quit['treat'].count()
print(attr_pefm)

# find the top/bottom 7% of control
control_sorted = data_pefm[data_pefm['treat']==0].sort_values(by='performance')
upper_cut = int( len(control_sorted)*(1-attr_pefm) )
mean_c_low = control_sorted.iloc[:upper_cut]['performance'].mean()
mean_c = control_sorted['performance'].mean()
print(mean_c_low , mean_c)
mean_t = data_pefm[data_pefm['treat']==1]['performance'].mean()
itt_upper = mean_t - mean_c_low

lower_cut = int( len(control_sorted)*attr_pefm )
mean_c_high = control_sorted.iloc[lower_cut:]['performance'].mean()
itt_lower = mean_t - mean_c_high

print('Lee bounds: ', itt_lower, ',', itt_upper)

0.8405707196029777
2.8055555555555554 3.2584434654919234
Lee bounds:  -0.771513353115727 , 0.42293109132871765


**Practical significance**: The upper is around 0.42 points. So the upper bound means in the best case, the employee's performance would increase once HWFH is in effect. The lower bound is around 77 points. That means each HWFH reduces the performance by 77 percentage point in the worst scenario. So after accounting for differential attrition in this situation, the experiment is still uninformative in the practical scene since the confidence interval is too large to determine the effect of HWFH on performance. Further investigation is needed.